In [33]:
import threading

class Counting:
    count = 0
    
    def count_down(self, n=0):
        self.count += 1
        print(self.count)

In [34]:
c = Counting()
for _ in range(100):
    c.count_down()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [49]:
import concurrent.futures

c = Counting()
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    executor.map(c.count_down, [_ for _ in range(100)]) 

1
2
3
45
6

789
10


11
12
131415
16
17
18


1920
21
22
23
24
25
26

27
28
29
30
31
32
33
34
35
3637
38
39

40
41
42
43
44
45
46
47
48
4950
51
52
53
54
55
56
57
58
59
60

61
62
63
64
65
66
67
6869
70
71
72
73
74
75
76
77
78
79
80
81

82
83
84
85
8687
88
89
90
91
92
93
94
95
96
97
98
99
100



In [170]:
import concurrent.futures
import requests
import threading
import time
import logging
import random

logging.basicConfig(level=logging.INFO,
                    format='(%(threadName)-0s) %(message)s',)

counter = 0

def increment_counter(fake_value):
    global counter
    logging.debug(f'thread_local counter is: {counter}')
    
    counter += 1
    
    logging.info(f'counter: {counter}')
    logging.info(f'counter: {counter}')
    print('counter:', counter)
    # logging.info(f'counter is: {counter}')


fake_data = [x for x in range(10)]

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(increment_counter, fake_data)

(ThreadPoolExecutor-146_0) thread_local counter is: 0
(ThreadPoolExecutor-146_0) counter: 1
(ThreadPoolExecutor-146_0) counter: 1
(ThreadPoolExecutor-146_1) thread_local counter is: 1
(ThreadPoolExecutor-146_0) thread_local counter is: 1
(ThreadPoolExecutor-146_1) counter: 2
(ThreadPoolExecutor-146_1) counter: 3
(ThreadPoolExecutor-146_0) counter: 3
(ThreadPoolExecutor-146_1) thread_local counter is: 3
(ThreadPoolExecutor-146_0) counter: 3
(ThreadPoolExecutor-146_1) counter: 4
(ThreadPoolExecutor-146_0) thread_local counter is: 4
(ThreadPoolExecutor-146_1) counter: 4
(ThreadPoolExecutor-146_0) counter: 5
(ThreadPoolExecutor-146_1) thread_local counter is: 5
(ThreadPoolExecutor-146_1) counter: 6
(ThreadPoolExecutor-146_1) counter: 6
(ThreadPoolExecutor-146_0) counter: 5
(ThreadPoolExecutor-146_1) thread_local counter is: 6
(ThreadPoolExecutor-146_0) thread_local counter is: 6
(ThreadPoolExecutor-146_1) counter: 7
(ThreadPoolExecutor-146_0) counter: 8
(ThreadPoolExecutor-146_1) counter: 

counter: 1
counter: 3
counter: 4
counter: 5
counter: 6
counter: 6
counter: 8
counter: 8
counter: 10
counter: 10


In [79]:
import threading
import logging
import random

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-0s) %(message)s',)

def show(d):
    try:
        val = d.val
    except AttributeError:
        logging.debug('No value yet')
    else:
        logging.debug('value=%s', val)

def f(d):
    show(d)
    d.val = random.randint(1, 100)
    show(d)

if __name__ == '__main__':
    d = threading.local()
    show(d)
    d.val = 999
    show(d)

    for i in range(2):
        t = threading.Thread(target=f, args=(d,))
        t.start()

(MainThread) No value yet
(MainThread) value=999
(Thread-7) No value yet
(Thread-7) value=31
(Thread-8) No value yet
(Thread-8) value=8


`d = threading.local()` marks the variable `d` as a `local` variable for each `thread`. When a new thread starts, it will create a **new instance** and use that instance for whatever processes require `d`.

```python
    d = threading.local() # => this create a new local instance in the main thread
    show(d) # No value yet
    d.val = 999 # assign value
    show(d) # value=999
```

When a thread `t` is created and starts, it will create it own variable `d`. Therefore when we `show(d)`, we don't have any value assign to `d.val` yet. `d` even doesn't have attr `val`, which catched by our `try catch` block to prompt the message `No value yet`

```python
    try:
        val = d.val
    except AttributeError:
        logging.debug('No value yet') # We see this message because val is not an attribute of d
    else:
        logging.debug('value=%s', val)
```

Only when we assign a value to `d.val`, we can `show(d)`

```python
def f(d):
    show(d)
    d.val = random.randint(1, 100) # assigning value
    show(d) # showing value
```

In [19]:
import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    print(f'############### Downloading from {url} ###############')
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
    print(f'############### Downloading done from {url} ###############')


def cdownload_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(download_site, sites)

def download_all_sites(sites):
    for site in sites:
        download_site(site)

if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    cdownload_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ############################## Downloading from http://olympus.realpython.org/dice ###############

############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ########

Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/diceRead 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############

############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/diceRead 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############

############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realp

In [27]:
import asyncio
import random
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-0s)%(message)s ',)

counter = 0
def log(msg):
    current_task = asyncio.tasks.current_task().get_name()
    logging.debug(f'[{current_task}]{msg}')
    
async def increment_counter():
    global counter
    log(f'Initial couter: {counter}')
    sleep_time = random.randint(1, 3)
    log(f'Going to sleep for {sleep_time} seconds')
    # We could wait here
    await asyncio.sleep(sleep_time)
    # After resume, need to make sure we increase counter and print it before other thread could do the same.
    # This guarantees we have a continuous counter with not duplication or missing value.
    counter += 1
    log(f'counter is: {counter}')
    # Two statements above will run without any interuption
    # We could wait afterward, and wrapping up this call later
    await asyncio.sleep(1)
    log('Resume and exit')
    
async def main():
    for _ in range(10):
        loop = asyncio.get_running_loop()
        loop.create_task(increment_counter())
        

loop = asyncio.get_running_loop()
loop.create_task(main())

    

<Task pending name='Task-214' coro=<main() running at <ipython-input-27-c3438ea24824>:29>>

(MainThread) [Task-215]Initial couter: 0
(MainThread) [Task-215]Going to sleep for 2 seconds
(MainThread) [Task-216]Initial couter: 0
(MainThread) [Task-216]Going to sleep for 2 seconds
(MainThread) [Task-217]Initial couter: 0
(MainThread) [Task-217]Going to sleep for 3 seconds
(MainThread) [Task-218]Initial couter: 0
(MainThread) [Task-218]Going to sleep for 1 seconds
(MainThread) [Task-219]Initial couter: 0
(MainThread) [Task-219]Going to sleep for 2 seconds
(MainThread) [Task-220]Initial couter: 0
(MainThread) [Task-220]Going to sleep for 3 seconds
(MainThread) [Task-221]Initial couter: 0
(MainThread) [Task-221]Going to sleep for 2 seconds
(MainThread) [Task-222]Initial couter: 0
(MainThread) [Task-222]Going to sleep for 1 seconds
(MainThread) [Task-223]Initial couter: 0
(MainThread) [Task-223]Going to sleep for 3 seconds
(MainThread) [Task-224]Initial couter: 0
(MainThread) [Task-224]Going to sleep for 3 seconds
(MainThread) [Task-218]counter is: 1
(MainThread) [Task-222]counter is

In [2]:
import concurrent.futures
import requests
import threading
import time
import asyncio

thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


async def download_site(url):
    # Making a function async is not enough, you need to put a trap somewhere
    # to make the task switch happen. Otherwise, this function will run until
    # it finishes, as a normal function does.
    print(f'############### Downloading from {url} ###############')
    session = get_session()
    
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
    print(f'############### Downloading done from {url} ###############')

async def download_all_sites(sites):
    loop = asyncio.get_running_loop()
    for site in sites:
        # to call another async function, we use await
         loop.create_task(download_site(site))
            
    print('download_all_sites has finished')

if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    loop = asyncio.get_running_loop()
    await loop.create_task(download_all_sites(sites))
    
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

download_all_sites has finished
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from

Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://w

Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://w

Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
Read 10267 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
############### Downloading from https://w

Eventhough we make our functions `download_site` and `download_all_sites` coroutine (by specifying `async` in function definition and run `download_all_sites` in an event loop:

```python
async def download_site(url):
    ...

async def download_all_sites(sites):
    ...
        
loop = asyncio.get_running_loop()
await loop.create_task(download_all_sites(sites))
```

The result, however, still remains approximately to that we got from running in one thread:
```
Downloaded 160 in 27.92220950126648 seconds
```

This is because deepdown in our program, we still have to wait for each http request to be done:
```python
with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
```
Let's break it down.
Our code will start here:
```python
loop = asyncio.get_running_loop() # Get the running loop
await loop.create_task(download_all_sites(sites)) # create a task to download all sites and wait for it to be done
```
This will start `download_all_sites` and it will actually create a task to download each site:
```python
async def download_all_sites(sites):
    loop = asyncio.get_running_loop()
    for site in sites:
        # to call another async function, we use await
         loop.create_task(download_site(site))
```            
As we specify `await` when calling `download_all_sites`, our execution will wait until this task completes.
```python
await loop.create_task(download_all_sites(sites))
```
It's okay to wait for the main task to be finished, we could create the task and let it run without waiting to it by emitting `await`.

Try it out and you will immediately see the `print(f"Downloaded {len(sites)} in {duration} seconds")` get executed right away:
```
Downloaded 160 in 0.0 seconds
```
Then, bunch of tasks will be running. Guess what, they are our `download_site` tasks. However, the manner these tasks run is still sequential. The program needs to finish downloading one site before moving the the next one. That is why our run time is almost same with running the program sequentially. Further inspection of the `download_site` function will explain our issue:
```python
async def download_site(url):
    # Making a function async is not enough, you need to put a trap somewhere
    # to make the task switch happen. Otherwise, this function will run until
    # it finishes, as a normal function does.
    print(f'############### Downloading from {url} ###############')
    session = get_session()
    
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
    print(f'############### Downloading done from {url} ###############')
```
Not many statements get executed, we can easily spot the bottle neck
```python
with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
```
Whenever a `download_site` task gets executed, it will come to this part and wait until `session.get(url)` finishes. By waiting, I mean the normal waiting where we need to wait for I/O operation, the CPU will sit there for a few seconds, doing nothing.
So, all of our effort to get the program running asynchornously up to this time is essentially wasted. The next idea is to try to start `session.get()` and somehow stop waiting for the response but run the other tasks instead. We only expect to return to this task once `session.get()` is done, we have our `response` and continue to finish the task by printing out `response`.

In [8]:
import concurrent.futures
import requests
import threading
import time
import asyncio
import aiohttp
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-0s) %(message)s',)

async def download_site(session, url):
    # Making a function async is not enough, you need to put a trap somewhere
    # to make the task switch happen. Otherwise, this function will run until
    # it finishes, as a normal function does.
    print(f'############### Downloading from {url} ###############')    
    async with session.get(url) as response:
        print(f"Read {response.content_length} from {url}")
    print(f'############### Downloading done from {url} ###############')

async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        loop = asyncio.get_running_loop()
        tasks = []
        for site in sites:
            # to call another async function, we use await
             tasks.append(loop.create_task(download_site(session, site)))
        pending = asyncio.gather(*tasks, return_exceptions=False)
        print(type(pending))
        await asyncio.wait(tasks)
    print('download_all_sites has finished')

if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    loop = asyncio.get_running_loop()
    await loop.create_task(download_all_sites(sites))
    
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

(MainThread) _GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=RuntimeError('Session is closed')>
Traceback (most recent call last):
  File "<ipython-input-6-3c0e3e8d62e3>", line 17, in download_site
    async with session.get(url) as response:
  File "c:\users\thaia\appdata\local\programs\python\python38\lib\site-packages\aiohttp\client.py", line 1012, in __aenter__
    self._resp = await self._coro
  File "c:\users\thaia\appdata\local\programs\python\python38\lib\site-packages\aiohttp\client.py", line 357, in _request
    raise RuntimeError('Session is closed')
RuntimeError: Session is closed


<class 'asyncio.tasks._GatheringFuture'>
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloading from https://www.jython.org ###############
############### Downloading from http://olympus.realpython.org/dice ###############
############### Downloa

Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from http

Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 3549 from https://www.jython.org
############### Downloading done from https://www.jython.org ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.realpython.org/dice ###############
Read 276 from http://olympus.realpython.org/dice
############### Downloading done from http://olympus.r

```
Downloaded 160 in 0.9356372356414795 seconds
```
What an improvement, this is even faster then multi threads execution. But why?
First, let's review some differences from our last version:
## download_all_sites
Our `session` now comes from a different library, `aiohttp`, and we only use 1 `session` object for all our task. In our new version, we use an async context manager to create our session:
```python
async with aiohttp.ClientSession() as session:
    ...
```
Notice that we need to use `async with` instead of a normal `with` because we're operating in the wolrd of async. A bare `with` will raise error:
```
<ipython-input-22-41745176ef51> in download_all_sites(sites)
     16 
     17 async def download_all_sites(sites):
---> 18     with aiohttp.ClientSession() as session:
     19         loop = asyncio.get_running_loop()
     20         tasks = []

c:\users\thaia\appdata\local\programs\python\python38\lib\site-packages\aiohttp\client.py in __enter__(self)
    961 
    962     def __enter__(self) -> None:
--> 963         raise TypeError("Use async with instead")
    964 
    965     def __exit__(self,

TypeError: Use async with instead
```
We also record all the tasks created in `tasks`
```python
tasks.append(loop.create_task(download_site(session, site))) # add tasks to our list to keep track of
```
and later on wait for all these tasks to be completed inside the context manager:
```
await asyncio.gather(*tasks, return_exceptions=False)
```
Without waiting, the execution will exit the context manager, closes the `session` in process and our `download_site` tasks don't have the required `session` to execute.
## download_site
The slight change in this function is that we will use the `session` object that is passed in, instead of calling `get_session()`. Another minor change in the `response` api of `aiohttp` is that `response.content` is now a `stream` so doesn't support `len()`. Rather, we could use `response.content_length`. After all, it's only a minor change to get our code work. The last change is the use of `async with`, asynchronous context manager.


Just like threading, `asyncio` creates all tasks, schedules and runs them. When each task gets to the point it need to wait for I/O, `session.get(url)` in this case, instead of waiting for the response, the task will now get suspended so other tasks could run (to the point those tasks need to wait for I/O and get suspended again). Until the reponses arrive, those tasks will again, one by one, pick up the responses and finish their execution.
![](https://files.realpython.com/media/Asyncio.31182d3731cf.png)

Reference: https://realpython.com/python-concurrency/

Basically, we've fully ultilized our CPU, minimalized waiting time (wasted time) by doing multiple http requests at the same time. (thanks that the server serves those requests concurrently (pretty much in parallel manner) as well, if not, we'll will be totally wasted. And of course, thanks to the `aiohttp` library)

In the end, asynchronous program requires all of its components run, well, asynchronously and the very component at the end of the stream running in parallel. Otherwise, the whole thing will be running in sequential manner a.k.a bottle necked.

In [39]:
import random

async def simple_coro(name):
    print('Greeting', name)
    await asyncio.sleep(random.randint(1, 3))
    print('Task is about to finish')
    return 'Data for ' + str(name)
   
async def main():
    loop = asyncio.get_running_loop()
    names = ['Alan', 'Bella', 'Cyde', 'Dane']
    tasks = []
    for name in names:
        print('creating task for', name)
        tasks.append(loop.create_task(simple_coro(name)))
    # results = await asyncio.gather(*tasks)
    # print(results)
    finish, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
    print(type(finish))
    print(finish)
    for item in finish:
        print('######################################')
        print('Item:', item)
        print('Type:', type(item))
        print('Result:', item.result())
        print('######################################')
        
    for item in pending:
        print('######################################')
        print('Item:', item)
        print('Type:', type(item))
        print('Status:', item.add_done_callback(lambda x: print('Special finish', x)))
        print('######################################')

loop = asyncio.get_running_loop()
task = loop.create_task(main())
await task
print('All tasks finished')

creating task for Alan
creating task for Bella
creating task for Cyde
creating task for Dane
Greeting Alan
Greeting Bella
Greeting Cyde
Greeting Dane
Task is about to finish
Task is about to finish
<class 'set'>
{<Task finished name='Task-1788' coro=<simple_coro() done, defined at <ipython-input-39-5f2c8dbbff72>:3> result='Data for Cyde'>, <Task finished name='Task-1789' coro=<simple_coro() done, defined at <ipython-input-39-5f2c8dbbff72>:3> result='Data for Dane'>}
######################################
Item: <Task finished name='Task-1788' coro=<simple_coro() done, defined at <ipython-input-39-5f2c8dbbff72>:3> result='Data for Cyde'>
Type: <class '_asyncio.Task'>
Result: Data for Cyde
######################################
######################################
Item: <Task finished name='Task-1789' coro=<simple_coro() done, defined at <ipython-input-39-5f2c8dbbff72>:3> result='Data for Dane'>
Type: <class '_asyncio.Task'>
Result: Data for Dane
######################################
#

asyncio.wait() vs. asyncio.wait_for()

asyncio.wait() will wait for 1 aw while wait_for() waits for multiples awaitable



wait_for() can wait for a coroutine

In [22]:
import asyncio

async def wait_for_me(name):
    print('start waiting')
    await asyncio.sleep(2)
    print('Thank you for waiting')
    
    return 'bye bye ' + name

try:
    result = await asyncio.wait_for(wait_for_me("Name"), timeout=10)
except asyncio.TimeoutError as e:
    print("Timeout, stop waiting")
print("Result:", result)

start waiting
Thank you for waiting
Result: bye bye Name


or a future, e.g. a task

In [18]:
import asyncio

async def wait_for_me(name):
    print('start waiting')
    await asyncio.sleep(2)
    print('Thank you for waiting')
    
    return 'bye bye ' + name

result = ""
loop = asyncio.get_running_loop()
task = loop.create_task(wait_for_me("Name"))
try:
    result = await asyncio.wait_for(task, timeout=5)
except asyncio.TimeoutError as e:
    print("Timeout, stop waiting")
    
print("Result:", result)

start waiting
Thank you for waiting
Result: bye bye Name


output of `wait_for` will be the returned value from future

mean whiles, `wait` can only wait for a list of futures. Neither `coroutine` nor a **single** `task` is accepted

In [29]:
import asyncio

async def wait_for_me(name):
    print('start waiting')
    await asyncio.sleep(2)
    print('Thank you for waiting')
    
    return 'bye bye ' + name

result = ""
try:
    done, pending = await asyncio.wait(wait_for_me("Name"), timeout=1)
except asyncio.TimeoutError as e:
    print("Timeout, stop waiting")
    
print("Done:", done)
print("Pending:", pending)

TypeError: expect a list of futures, not coroutine

In [27]:
import asyncio

async def wait_for_me(name):
    print('start waiting')
    await asyncio.sleep(2)
    print('Thank you for waiting')
    
    return 'bye bye ' + name

result = ""
loop = asyncio.get_running_loop()
task = loop.create_task(wait_for_me("Name"))
try:
    done, pending = await asyncio.wait(task, timeout=1)
except asyncio.TimeoutError as e:
    print("Timeout, stop waiting")
    
print("Done:", done)
print("Pending:", pending)

TypeError: expect a list of futures, not Task

start waiting
Thank you for waiting


In [33]:
import asyncio

async def wait_for_me(name):
    print('start waiting')
    await asyncio.sleep(2)
    print('Thank you for waiting')
    
    return 'bye bye ' + name

result = ""
loop = asyncio.get_running_loop()
task = loop.create_task(wait_for_me("Name"))
try:
    done, pending = await asyncio.wait([task], timeout=10)
except asyncio.TimeoutError as e:
    print("Timeout, stop waiting")
    
print("Done:", done)
print("Pending:", pending)

for task in done:
    print(task.result())

start waiting
Thank you for waiting
Done: {<Task finished name='Task-57' coro=<wait_for_me() done, defined at <ipython-input-33-1a17764ce41e>:3> result='bye bye Name'>}
Pending: set()
bye bye Name


Since `wait` accepts a list of futures, and allows to set a timeout `timeout=`, its returned value are also some sort of a list of items. In fact, it returns 2 lists: a list of completed tasks and a list of uncompleted tasks.

To retrieve the result of each task, call `task.result()`